In [22]:
# Import standard libraries
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import random
import numpy as np
import yfinance as yf # ticker data 
from pandas_datareader import data as pdr # market proxy and treasury rates data
import datetime as dt
from numpy import nan
from time import sleep
import concurrent.futures
from functools import reduce
from scipy.stats import norm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import scipy 
import time

In [23]:
# Fetch historical price data for the desired asset using our data
company_dataset = pd.read_csv("Data/CRSP_stocks_clean.csv")

In [24]:
company_dataset

,Unnamed: 0,date,TICKER,CUSIP,PRC,RET,VOL,BID,ASK,SHROUT
0,1718,2000-01-03,AIR,36110,17.5625,-0.020906,109000.0,17.375,17.750,27401.0
1,10251,2000-01-04,AIR,36110,17.6250,0.003559,51700.0,17.375,17.875,27401.0
2,18784,2000-01-05,AIR,36110,17.8125,0.010638,70400.0,17.500,18.000,27401.0
3,27320,2000-01-06,AIR,36110,17.9375,0.007018,60100.0,17.750,18.000,27401.0
4,35852,2000-01-07,AIR,36110,18.5625,0.034843,148000.0,18.125,18.750,27401.0
...,...,...,...,...,...,...,...,...,...,...
8109235,42531451,2022-12-23,TK,Y8564W10,4.5800,-0.002179,422897.0,4.580,4.590,101872.0
8109236,42540998,2022-12-27,TK,Y8564W10,4.5700,-0.002183,664421.0,4.570,4.580,101872.0
8109237,42550538,2022-12-28,TK,Y8564W10,4.3100,-0.056893,1103593.0,4.310,4.320,101872.0
8109238,42560071,2022-12-29,TK,Y8564W10,4.4300,0.027842,764632.0,4.420,4.430,101872.0


In [25]:
# Group the data by the ticker symbol
grouped_data = company_dataset.groupby('TICKER')
print(grouped_data)

In [26]:
# Function to calculate Expected Shortfall
def calculate_es(returns, alpha=0.05):
    sorted_returns = np.sort(returns)
    index = int(alpha * len(sorted_returns))
    return -np.mean(sorted_returns[:index])

In [27]:
# Calculate the Expected Shortfall for each ticker
expected_shortfalls = {}
for ticker, group in grouped_data:
    returns = group['RET'].dropna()  # Assuming 'ret' column contains daily returns
    es = calculate_es(returns)
    expected_shortfalls[ticker] = es

C:\Users\joshl\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\joshl\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [28]:
# Print the results
for ticker, es in expected_shortfalls.items():
    print(f"Ticker: {ticker}, Expected Shortfall (ES): {es}")

Ticker: A, Expected Shortfall (ES): 0.060375418685121106
Ticker: AA, Expected Shortfall (ES): 0.06232240284360189
Ticker: AAIC, Expected Shortfall (ES): 0.04019544444444444
Ticker: AAME, Expected Shortfall (ES): 0.11004814878892734
Ticker: AAN, Expected Shortfall (ES): 0.056887068493150686
Ticker: AAON, Expected Shortfall (ES): 0.055666629757785475
Ticker: AAPL, Expected Shortfall (ES): 0.05575637024221453
Ticker: AAS, Expected Shortfall (ES): 0.06697745000000001
Ticker: AB, Expected Shortfall (ES): 0.0580071179245283
Ticker: ABC, Expected Shortfall (ES): 0.039817048507462684
Ticker: ABCB, Expected Shortfall (ES): 0.0662597197231834
Ticker: ABEV, Expected Shortfall (ES): 0.04981795652173914
Ticker: ABFS, Expected Shortfall (ES): 0.07069628888888889
Ticker: ABIO, Expected Shortfall (ES): 0.11248366285714285
Ticker: ABM, Expected Shortfall (ES): 0.04830694117647058
Ticker: ABV, Expected Shortfall (ES): 0.07275368674698796
Ticker: ABX, Expected Shortfall (ES): 0.058673273109243695
Ticker: